In [3]:
# PERFORM TEXT VECTORIZATION...

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\cyber bulling dataset\data.csv")

# FUNCTION FOR PREPROCESSING
def preprocess_text(text):
    if isinstance(text, str):
        # REMOVE URLS
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)

        # CONVERT TO LOWER CASE
        text = text.lower()

        # REMOVE SPECIAL CHARACTERS
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)

        return text
    elif isinstance(text, float) and np.isnan(text):
        return ""
    else:
        return str(text)

# SELECT TEXT COLUMN FOR PROCESS
text_columns = [
    "Email Address",
    "What do you consider as Cyber bullying?",
    "If Yes , What was the way you dealt with the same",
    "What do you think made you the victim of cyber bullying?",
    "What Platform according to you leads in cyber bullying",
    "What type of people are usually the bullies on the internet?",
    "What lead you to bully someone ? ( if yes )",
    "What kind of people are usually the victims of cyber-bullying?",
    "What are the causes of cyber bullying according to you",
    "Who are bullies usually?",
    "Youtube Roasting is a form of bullying. Do you agree ?",
]

# PREPROCESS TEXT
for column in text_columns:
    df[column] = df[column].apply(preprocess_text)

# TEXT VECTORIZATION
vectorizer = TfidfVectorizer()
text_data = df[text_columns].apply(lambda x: ' '.join(x), axis=1)
vectorized_text = vectorizer.fit_transform(text_data)

# GET FEATURE NAME
feature_names = vectorizer.vocabulary_

# CONVERT THE VECTORIZED TEXT TO A DATA FRAME
vectorized_df = pd.DataFrame(vectorized_text.toarray(), columns=feature_names)

# CONCATENATE VACTORIZED DATAFRAME WITH THE ORGINAL DATAFRAME
df = pd.concat([df, vectorized_df], axis=1)

print(df.head())


            Timestamp               Email Address What's Your Age?  \
0  1/22/2022 11:39:31      armadhav2007 gmail com          15 - 18   
1  1/22/2022 11:40:39  nikhilkumar38674 gmail com         Above 18   
2  1/22/2022 11:41:15      guptasayyam7 gmail com          15 - 18   
3  1/22/2022 11:41:57      sonaissuhani gmail com          15 - 18   
4  1/22/2022 11:42:02        madmaxhu69 gmail com          15 - 18   

             What do you consider as Cyber bullying?  \
0  hacking   phishing your private details  spamm...   
1  hacking   phishing your private details  spamm...   
2  spamming   unlawfully morphing your images  me...   
3  mean inappropriate comments  pranks calls by s...   
4  mean inappropriate comments  spamming   unlawf...   

  What Age group of people are most Vulnerable to Cyber bullying  \
0                                              19-30               
1                                 equally vulnerable               
2                                     

In [6]:
#PERFORM RNNs MODEL TO PREDICT-What kind of people are usually the victims of cyber-bullying?

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder


# SEED FOR RANDAOM NUMBER GENERATOR IN NUMPY
np.random.seed(42)

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\cyber bulling dataset\data.csv")


# FUNCTION FOR PREPROCESSING

def preprocess_text(text):
    if isinstance(text, str):
        
        # REMOVE URLS
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)

        # CONVERT TO LOWER CASE
        text = text.lower()

        # REMOVE SPECIAL CHARACTERS
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)

        return text
    elif isinstance(text, float) and np.isnan(text):
        return ""
    else:
        return str(text)

    
    
    
# SELECT TEXT COLUMN FOR PROCESS

text_columns = [
    "Email Address",
    "What do you consider as Cyber bullying?",
    "If Yes , What was the way you dealt with the same",
    "What do you think made you the victim of cyber bullying?",
    "What Platform according to you leads in cyber bullying",
    "What type of people are usually the bullies on the internet?",
    "What lead you to bully someone ? ( if yes )",
    "What kind of people are usually the victims of cyber-bullying?",
    "What are the causes of cyber bullying according to you",
    "Who are bullies usually?",
    "Youtube Roasting is a form of bullying. Do you agree ?",
]

# PREPROCESS TEXT

for column in text_columns:
    df[column] = df[column].apply(preprocess_text)

# Split the data into training and testing sets
X = df[text_columns].apply(lambda x: ' '.join(x), axis=1)
y = df['What kind of people are usually the victims of cyber-bullying?'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TOKENIZE THE TEXT DATA

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_tokens = tokenizer.texts_to_sequences(X_train)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

# PAD SEQUENCE TO HAVE THE SAME LENGTH

max_sequence_length = max(len(seq) for seq in X_train_tokens)
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_sequence_length)

# CONVERT TARGERT VARIABLE TO INTEGER

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
num_classes = len(label_encoder.classes_)

# ONE HOT ENCODEING TO TARGET VARIABLE
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes=num_classes)

# RNN MODEL
embedding_dim = 100  # Dimensionality of the word embeddings
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# TRAIN THE MODEL
model.fit(X_train_pad, y_train_encoded, validation_data=(X_test_pad, y_test_encoded), epochs=100, batch_size=32)

# EVALUATE
loss, accuracy = model.evaluate(X_test_pad, y_test_encoded, verbose=0)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/100
2/2 [==============================] - 3s 671ms/step - loss: 1.3797 - accuracy: 0.5556 - val_loss: 1.3466 - val_accuracy: 0.7143
Epoch 2/100
2/2 [==============================] - 0s 154ms/step - loss: 1.3311 - accuracy: 0.7963 - val_loss: 1.3016 - val_accuracy: 0.7143
Epoch 3/100
2/2 [==============================] - 0s 151ms/step - loss: 1.2712 - accuracy: 0.7963 - val_loss: 1.2363 - val_accuracy: 0.7143
Epoch 4/100
2/2 [==============================] - 0s 151ms/step - loss: 1.1823 - accuracy: 0.7963 - val_loss: 1.1247 - val_accuracy: 0.7143
Epoch 5/100
2/2 [==============================] - 0s 137ms/step - loss: 1.0225 - accuracy: 0.7963 - val_loss: 0.8957 - val_accuracy: 0.7143
Epoch 6/100
2/2 [==============================] - 0s 142ms/step - loss: 0.7954 - accuracy: 0.7963 - val_loss: 0.9460 - val_accuracy: 0.7143
Epoch 7/100
2/2 [==============================] - 0s 143ms/step - loss: 0.6952 - accuracy: 0.7963 - val_loss: 0.8401 - val_accuracy: 0.7143
Epoch 8/100
2

In [19]:
#PEFROM RNNs MODEL TO PREDICT MULTIPLE  TARGET VARIABLE .....

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

#SEED FOR RANDAOM NUMBER GENERATOR IN NUMPY

np.random.seed(42)

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\cyber bulling dataset\data.csv")


# FUNCTION


def preprocess_text(text):
    if isinstance(text, str):
        # REMOVE URLS
        text = re.sub(r"http\S+|www\S+|https\S+", "", text )

        # CONVERT TO LOWER CASE
        text = text.lower()

        # REMOVE SPECIAL CHARACTERS
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)

        return text
    elif isinstance(text, pd.Series):
        # Apply the preprocess_text function to each element in the Series
        return text.apply(lambda x: preprocess_text(x))
    else:
        return ""


# SELECT TEXT COLUMN FOR PROCESS
text_columns = [
    "Email Address",
    "What do you consider as Cyber bullying?",
    "If Yes , What was the way you dealt with the same",
    "What do you think made you the victim of cyber bullying?",
    "What Platform according to you leads in cyber bullying",
    "What type of people are usually the bullies on the internet?",
    "What lead you to bully someone ? ( if yes )",
    "What kind of people are usually the victims of cyber-bullying?",
    "What are the causes of cyber bullying according to you",
    "Who are bullies usually?",
    "Youtube Roasting is a form of bullying. Do you agree ?",
]

# PREPROCESS THE TEXT
for column in text_columns:
    df[column] = preprocess_text(df[column])
    

# SPLIT DATA TRAINING AND TESTING SET
X = df[text_columns].apply(lambda x: ' '.join(x), axis=1)
y = df[text_columns]

# TOKENIZE THE DATA
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# PAD SEQUENCE THE INPUT DATA
max_input_sequence_length = 100  
X_pad = pad_sequences(X_seq, maxlen=max_input_sequence_length)



# ONE HOT ENCODING TO TARGET VARIABLE


label_encoders = {}
y_pad = []
for column in text_columns:
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y[column])
    y_pad.append(to_categorical(y_encoded))
    label_encoders[column] = label_encoder

    
    
# RNN MODEL WITH MULTIPLE LAYER


vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  # Adjust as needed

models = []
for i, column in enumerate(text_columns):
    num_classes = len(label_encoders[column].classes_)
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_input_sequence_length))
    model.add(LSTM(100))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    models.append(model)

    
    
# TRAIN THE MODEL


for i, model in enumerate(models):
    print(f"Training model for column: {text_columns[i]}")
    model.fit(X_pad, y_pad[i], validation_split=0.2, epochs=1000, batch_size=32)

# EVALUATE


for i, model in enumerate(models):
    loss, accuracy = model.evaluate(X_pad, y_pad[i], verbose=0)
    print(f'Test Loss for column {text_columns[i]}: {loss:.4f}')
    print(f'Test Accuracy for column {text_columns[i]}: {accuracy:.4f}')


Training model for column: Email Address
Epoch 1/1000
2/2 [==============================] - 5s 1s/step - loss: 4.2063 - accuracy: 0.0185 - val_loss: 4.2148 - val_accuracy: 0.0000e+00
Epoch 2/1000
2/2 [==============================] - 0s 181ms/step - loss: 4.1992 - accuracy: 0.0185 - val_loss: 4.2236 - val_accuracy: 0.0000e+00
Epoch 3/1000
2/2 [==============================] - 0s 180ms/step - loss: 4.1932 - accuracy: 0.0370 - val_loss: 4.2332 - val_accuracy: 0.0000e+00
Epoch 4/1000
2/2 [==============================] - 0s 176ms/step - loss: 4.1869 - accuracy: 0.0741 - val_loss: 4.2443 - val_accuracy: 0.0000e+00
Epoch 5/1000
2/2 [==============================] - 0s 162ms/step - loss: 4.1802 - accuracy: 0.0926 - val_loss: 4.2584 - val_accuracy: 0.0000e+00
Epoch 6/1000
2/2 [==============================] - 0s 133ms/step - loss: 4.1719 - accuracy: 0.0926 - val_loss: 4.2766 - val_accuracy: 0.0000e+00
Epoch 7/1000
2/2 [==============================] - 0s 133ms/step - loss: 4.1615 - acc

In [21]:
#MODEL INTERECT WITH USER.....

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Set seed for reproducibility
np.random.seed(42)

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\cyber bulling dataset\data.csv")

def preprocess_text(text):
    if isinstance(text, str):
        # REMOVE URLS
        text = re.sub(r"http\S+|www\S+|https\S+", "", text)

        # CONVERT TO LOWER CASE
        text = text.lower()

        # REMOVE SPECIAL CHARACTERS
        text = re.sub(r"[^a-zA-Z0-9]", " ", text)

        return text
    elif isinstance(text, pd.Series):
        # Apply the preprocess_text function to each element in the Series
        return text.apply(lambda x: preprocess_text(x))
    else:
        return ""

# SELECT TEXT COLUMN FOR PROCESS
text_columns = [
    "Email Address",
    "What do you consider as Cyber bullying?",
    "If Yes , What was the way you dealt with the same",
    "What do you think made you the victim of cyber bullying?",
    "What Platform according to you leads in cyber bullying",
    "What type of people are usually the bullies on the internet?",
    "What lead you to bully someone ? ( if yes )",
    "What kind of people are usually the victims of cyber-bullying?",
    "What are the causes of cyber bullying according to you",
    "Who are bullies usually?",
    "Youtube Roasting is a form of bullying. Do you agree ?",
]

# PREPROCESS THE TEXT
for column in text_columns:
    df[column] = preprocess_text(df[column])

# SPLIT DATA INTO TRAINING AND TESTING SET
X = df[text_columns].apply(lambda x: ' '.join(x), axis=1)
y = df[text_columns]

# TOKENIZE THE DATA
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

#PAD SEQUENCE FOR THE INPUT DATA
max_input_sequence_length = 100  # Adjust as needed
X_pad = pad_sequences(X_seq, maxlen=max_input_sequence_length)

# ONE HOT ENCODING FOR TARGET VARIABLE
label_encoders = {}
y_pad = []
for column in text_columns:
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y[column])
    y_pad.append(to_categorical(y_encoded))
    label_encoders[column] = label_encoder

# RNN MODEL WITH MULTIPLE OUTPUT LAYER
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  # Adjust as needed

models = []
for i, column in enumerate(text_columns):
    num_classes = len(label_encoders[column].classes_)
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_input_sequence_length))
    model.add(LSTM(100))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    models.append(model)

# TRAIN MODEL
for i, model in enumerate(models):
    print(f"Training model for column: {text_columns[i]}")
    model.fit(X_pad, y_pad[i], validation_split=0.2, epochs=100, batch_size=32)

# EVALUATE MODEL
for i, model in enumerate(models):
    loss, accuracy = model.evaluate(X_pad, y_pad[i], verbose=0)
    print(f'Test Loss for column {text_columns[i]}: {loss:.4f}')
    print(f'Test Accuracy for column {text_columns[i]}: {accuracy:.4f}')

# FUNCTION TO GET PREDICTION FROM MODEL
def get_predictions(text):
    # Preprocess the input text
    preprocessed_text = preprocess_text(text)
    
    # TOKENIZE PREPROCESS TEXT
    text_seq = tokenizer.texts_to_sequences([preprocessed_text])
    
    # PAD SEQUENCE
    text_pad = pad_sequences(text_seq, maxlen=max_input_sequence_length)
    
    # WILL GET PREDICTION FROM EACH MODEL
    predictions = {}
    for i, column in enumerate(text_columns):
        prediction = models[i].predict(text_pad)
        predicted_class = label_encoders[column].inverse_transform(np.argmax(prediction, axis=1))[0]
        predictions[column] = predicted_class
    
    return predictions

# INTERECTIVE LOOP   GET USER INPUT AND MAKE PREDICTION
while True:
    user_input = input("Enter your text: ")
    if user_input.lower() == "exit":
        break
    else:
        predictions = get_predictions(user_input)
        print("Predictions:")
        for column, predicted_class in predictions.items():
            print(f"{column}: {predicted_class}")


Training model for column: Email Address
Epoch 1/100
2/2 [==============================] - 4s 788ms/step - loss: 4.2068 - accuracy: 0.0000e+00 - val_loss: 4.2090 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 183ms/step - loss: 4.1997 - accuracy: 0.0185 - val_loss: 4.2152 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 184ms/step - loss: 4.1949 - accuracy: 0.0556 - val_loss: 4.2218 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 183ms/step - loss: 4.1891 - accuracy: 0.0741 - val_loss: 4.2291 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 190ms/step - loss: 4.1827 - accuracy: 0.0926 - val_loss: 4.2379 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 183ms/step - loss: 4.1758 - accuracy: 0.0926 - val_loss: 4.2492 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 207ms/step - loss: 4.1671 - acc